# Image_Classification_TensorFlow (Training)

<a class="anchor" id="0"></a>
# Table of Contents

1. [套件安裝與載入](#1)
1. [環境檢測與設定](#2)
1. [開發參數設定](#3)
1. [資料處理](#4)
    -  [載入CSV檔](#4.1)
    -  [檢查CSV檔缺失值](#4.2)
1. [定義模型方法](#5)
1. [定義回調函數方法](#6)
1. [製作資料集＆資料擴增&訓練模型](#7)
1. [混淆矩陣](#8)
1. [待辦事項](#9)

# 1. 套件安裝與載入<a class="anchor" id="1"></a>
[Back to Table of Contents](#0)

In [ ]:
!pip install -q efficientnet
import efficientnet.tfkeras as efn

In [ ]:
# !pip install tensorflow-addons
# import tensorflow_addons.optimizers as addons_optimizers

In [ ]:
# 資料處理套件
import os
import gc
import cv2
import random
import datetime
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from collections import Counter
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# 設定顯示中文字體
from matplotlib.font_manager import FontProperties
plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei'] # 用來正常顯示中文標籤
plt.rcParams['font.family'] = 'AR PL UMing CN'
plt.rcParams['axes.unicode_minus'] = False # 用來正常顯示負號

In [ ]:
# tensorflow深度學習模組套件
import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow.keras.layers as layers
import tensorflow.keras.losses as losses
import tensorflow.keras.callbacks as callbacks
import tensorflow.keras.optimizers as optimizers
import tensorflow.keras.applications as applications

from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.client import device_lib

# 2. 環境檢測與設定<a class="anchor" id="2"></a>
[Back to Table of Contents](#0)

In [ ]:
# 查看設備
print(device_lib.list_local_devices())

In [ ]:
# 查看tensorflow版本
print(tf.__version__)

# 查看圖像通道位置
print(K.image_data_format())

In [ ]:
'''執行環境參數設定'''

# (Boolean)是否為本機
LOCAL = True

# (Boolean)是否為 Colab
COLAB = False

# (String)CPU/GPU/TPU
DEVICE = "GPU"


'''檔案路徑參數設定'''

# (String)Root路徑
if LOCAL:
    PATH = r'../'
elif COLAB:
    PATH = r'/content/drive/My Drive/Colab Notebooks/'
else:
    PATH = r'../input/'
    OUTPUT_PATH = r'/kaggle/working/'
    
# (String)資料根路徑
DATA_ROOT_PATH = PATH+r'datasets/AI_CUP_2020_AIMango_Defective_Classification/' 

# (String)訓練資料路徑
TRAIN_DATA_PATH = DATA_ROOT_PATH+r'C1-P2_Train Dev/Train'

# (String)訓練CSV路徑，如為None則不讀CSV檔
TRAIN_CSV_PATH = DATA_ROOT_PATH+r'C1-P2_Train Dev/train.csv'

# (String)專案名稱
PROJECT_NAME = 'AI_CUP_2020_AIMango_Defective_Classification'

# (String)專案檔案儲存路徑
if LOCAL or COLAB:
    OUTPUT_PATH = PATH
PROJECT_PATH = OUTPUT_PATH+PROJECT_NAME+'/'+PROJECT_NAME+' '+datetime.datetime.now().strftime("%Y-%m-%d %H:%M")

# (String)權重名稱(使用哪個權重)
WEIGHTS_NAME = 'efficientnetb7'

# (String)模型名稱(使用哪個模型)
MODEL_NAME = 'efficientnetb7'

# (String)讀取預訓練權重的儲存路徑 
LOAD_WEIGHTS_PATH = PROJECT_PATH+r'/models/backup/'+WEIGHTS_NAME+'.h5'

# (String)讀取預訓練模型的儲存路徑 
LOAD_MODEL_PATH = PROJECT_PATH+r'/models/backup/'+MODEL_NAME+'.h5'

# (String)CSV的儲存路徑
CSV_SAVE_PATH = PROJECT_PATH+r'/csv/'+MODEL_NAME+'.csv'

# (String)TensorBoard logs的儲存路徑 %tensorboard --logdir logs/fit
TENSORBOARD_LOGS_PATH = PROJECT_PATH+r'/logs/fit'

# (String)訓練模型的儲存路徑
TRAIN_MODEL_PATH = PROJECT_PATH+r'/models/'+MODEL_NAME+'.h5'

In [ ]:
if not LOCAL and COLAB:
    from google.colab import drive
    drive.mount('/content/drive')

In [ ]:
if DEVICE != "CPU":
    !nvidia-smi

In [ ]:
if DEVICE == "TPU":
    print("connecting to TPU...")
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        print('Running on TPU ', tpu.master())
    except ValueError:
        print("Could not connect to TPU")
        tpu = None

    if tpu:
        try:
            print("initializing  TPU ...")
            tf.config.experimental_connect_to_cluster(tpu)
            tf.tpu.experimental.initialize_tpu_system(tpu)
            strategy = tf.distribute.experimental.TPUStrategy(tpu)
            print("TPU initialized")
        except _:
            print("failed to initialize TPU")
    else:
        DEVICE = "GPU"

if DEVICE != "TPU":
    print("Using default strategy for CPU and single GPU")
    strategy = tf.distribute.get_strategy()

if DEVICE == "GPU":
    print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
    

AUTO     = tf.data.experimental.AUTOTUNE
REPLICAS = strategy.num_replicas_in_sync
print(f'REPLICAS: {REPLICAS}')

In [ ]:
# 動態申請顯存
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)

In [ ]:
if os.path.isfile(TRAIN_CSV_PATH):
    LOAD_CSV = True
else:
    LOAD_CSV = False

In [ ]:
if not os.path.isdir(PROJECT_PATH+r'/models/'):
    os.makedirs(PROJECT_PATH+r'/models/')

# 3. 開發參數設定<a class="anchor" id="3"></a>
[Back to Table of Contents](#0)

In [ ]:
'''客製參數設定'''


'''資料參數設定'''

# (Int)分類數量
CLASSES = 3

# (Int)有CSV檔該參數才有用，1則為不做交叉驗證
FOLD = 1

# (Int)沒CSV檔，FOLD該參數固定為1
if not LOAD_CSV:
    FOLD = 1

# (Int)圖片尺寸
IMAGE_SIZE = [224]*FOLD

# (String)圖片副檔名
IMAGE_NAME_EXTENSION = '.jpg'

# (String)CSV圖片檔名欄位(不包含路徑)
IMAGE_NAME = 'image_id'

# (String)CSV圖片檔名欄位(包含路徑)
IMAGE_NAME_ROOT = 'image'

# (String)CSV標籤欄位
LABEL_NAME = 'grade'

# (String)CSV標籤欄位類型
LABEL_NAME_TYPE = 'string'

# (Boolean)CSV圖片檔名欄位是否包含副檔名
IMAGE_NAME_HAVE_EXTENSION = True

#  (String)預設：'rgb'，圖像是否轉換為 1 個或 3 個顏色通道
COLOR_MODE = 'rgb'

#  (String)預設：'categorical'，決定返回標籤數組的類型："categorical" 將是 2D one-hot 編碼標籤
CLASS_MODE = 'categorical'

# (Int)不同的種子會產生不同的Random或分層K-FOLD分裂, 42則是預設固定種子
SEED = 42

if FOLD == 1:
    # (Float)驗證集佔訓練集的比率，FOLD>1則不啟用
    DATA_SPLIT = 0.2
else:
    # (String)切分訓練集跟驗證集方式
    SKF = StratifiedKFold(n_splits=FOLD,shuffle=True,random_state=SEED)


'''資料擴增參數設定'''

# (Boolean)對輸入數據施加ZCA白化
ZCA_WHITENING = False

# (Float)數據提升時圖片隨機轉動的角度
ROTATION_RANGE = 15.0

# (Float)圖片寬度的某個比例，數據提升時圖片水平偏移的幅度
WIDTH_SHIFT_RANGE = 0.2

# (Float)圖片高度的某個比例，數據提升時圖片水平偏移的幅度
HEIGHT_SHIFT_RANGE = 0.2

# (Float)剪切強度（逆時針方向的剪切變換角度）
SHEAR_RANGE = 0.2

# (Float)隨機縮放的幅度
ZOOM_RANGE = 0.2

# (Boolean)進行隨機水平翻轉
HORIZONTAL_FILP = True

# (Boolean)進行隨機垂直翻轉
VERTICAL_FILP = False

# (String)預設︰'nearest'，當進行變換時超出邊界的點將根據本參數給定的方法進行處理
FILL_MODE = 'nearest'


''''模型參數設定'''

# (Boolean)使用TF模型，如為False則須客制另外撰寫
USE_BASE_MODEL = True

if USE_BASE_MODEL:
    # (Boolean)使用EFFICIENTNET模型
    USE_EFFICIENTNET_MODEL = True
    if USE_EFFICIENTNET_MODEL:
        # (Int List)使用哪一種EFFICIENTNET
        EFF_NET = [0]*FOLD

        # (Model List)列出每種縮放尺寸的EFFICIENTNET
        BASE_MODEL = [efn.EfficientNetB0, efn.EfficientNetB1, efn.EfficientNetB2, efn.EfficientNetB3, 
                efn.EfficientNetB4, efn.EfficientNetB5, efn.EfficientNetB6, efn.EfficientNetB7]
    else:
        # (Model)建立TF模型
        BASE_MODEL = applications.MobileNetV2

# (Boolean)是否使用TF權重
LOAD_TF_WEIGHTS = True

if LOAD_TF_WEIGHTS:
    # (String)TF預訓練權重為 imagenet/noisy-student
    WEIGHTS = 'imagenet'
else:
    WEIGHTS = None

# (Boolean)TF模型是否包含完全連接網路頂部的網路層
INCLUDE_TOP = False
    
# (Boolean)TF模型是否可訓練權重(不包括頂部網路層)
BASE_MODEL_TRAINABLE = True

# (Boolean)是否已有客製模型，僅載入權重
LOAD_WEIGHTS = False

# (Boolean)是否載入完整客製(模型+權重)
LOAD_MODEL = False

# (Float)Dropout比率 0.5
DROPOUT = 0.5

# 最後輸出時的激活函數，雙分類為sigmoid，多分類為softmax
ACTIVATION_FUNCTION = 'softmax'

# (Boolean)是否印出完整模型
MODEL_PRINT = False

# (Boolean)是否儲存模型圖表
SAVE_MODEL_DIAGRAM = True

if SAVE_MODEL_DIAGRAM:
    # (Boolean)是否儲存模型圖表網路形狀
    SAVE_MODEL_SHAPE = True
    
    # (String)儲存模型圖表完整檔名
    SAVE_MODEL_FILENAME = PROJECT_PATH+r'/models/'+MODEL_NAME+'.png'


''''回調函數參數設定'''

# (Boolean)回調函數 ModelCheckpoint / ParallelModelCheckpoint 是否啟用
CALLBACKS_CHECK_POINTER = True

# (Boolean)回調函數 EarlyStoppin 是否啟用
CALLBACKS_EARLY_STOPPING = False

# (Boolean)回調函數 CSVLogger 是否啟用
CALLBACKS_CSV_LOGGER = False

# (Boolean)回調函數 LearningRateScheduler 是否啟用
CALLBACKS_LR_SCHEDULER = True

# (Boolean)回調函數 ReduceLROnPlateau 是否啟用
CALLBACKS_REDUCE_LR = True

# (Boolean)回調函數 TensorBoard 是否啟用
CALLBACKS_TENSOR_BOARD = False

# (String)回調函數監控數值 acc/val_acc/loss/val_loss
MONITOR = 'val_loss'

# (Boolean)回調函數 ModelCheckpoint / ParallelModelCheckpoint 是否只儲存最佳模型 False
SAVE_BEST_ONLY = True

# (Boolean)回調函數 ModelCheckpoint / ParallelModelCheckpoint 是否只儲存權重 False
SAVE_WEIGHTS_ONLY = False

# (Int)回調函數 EarlyStopping 沒有改善的時期數，之後訓練將停止 10
PATIENCE_ELS = 10

# (Boolean)回調函數 CSVLogger True為是否接下去原有CSV檔，False為覆蓋 
APPEND = True

# (Float)回調函數 LearningRateSchedule 初始學習率 0.000003
LEARNING_RATE_START = 0.000003

# (Float)回調函數 LearningRateSchedule 最大學習率 0.000020
LEARNING_RATE_MAX = 0.000020

# (Float)回調函數 LearningRateSchedule 最小學習率 0.000001
LEARNING_RATE_MIN = 0.000001

# (Float)回調函數 LearningRateSchedule 學習率多少時期後開始上升 5
LEARNING_RATE_RAMPUP_EPOCHS = 5

# (Float)回調函數 LearningRateSchedule 學習率支撐時期 0
LEARNING_RATE_SUSTAIN_EPOCHS = 0

# (Float)回調函數 LearningRateSchedule 學習率衰減 0.8
LEARNING_RATE_EXP_DECAY = 0.8

# (Float)回調函數 ReduceLROnPlateau 被降低的因數。新的學習速率 = 學習速率 * 因數 0.2
FACTOR = 0.2

# (Int)回調函數 ReduceLROnPlateau 沒有進步的訓練輪數，在這之後訓練速率會被降低 5 
PATIENCE_RLR = 5

# (Float)回調函數 ReduceLROnPlateau 最小學習率 0
LEARNING_RATE_MIN_RLR = 0

# (String)回調函數 TensorBoard 'batch'或'epoch'或整數。使用時'batch'，每批之後將損失和指標寫入TensorBoard
# 同樣適用於'epoch'。如果使用整數，假設1000，回調將每1000批將指標和損失寫入TensorBoard
UPDATE_FREQ = 'epoch'

# (Boolean)回調函數 TensorBoard 是否在TensorBoard中可視化圖形。當write_graph設置為True時，日誌文件可能會變得很大
WRITE_GRAPH = True

# (Int/String)回調函數 TensorBoard 分析批次以採樣計算特徵。profile_batch必須是非負整數或整數元組。一對正整數表示要分析的批次範圍
# 默認情況下，它將配置第二批。將profile_batch = 0設置為禁用分析 5 / '10,20'
PROFILE_BATCH = 5


''''編譯參數設定'''

if not CALLBACKS_LR_SCHEDULER:
    # (Float)優化器學習率 1e-3/1e-1
    LEARNING_RATE = None
    
    # (Float)優化器權重衰減 5e-5/5e-4
    WEIGHT_DECAY = None

# (Float)加速優化器在相關方向上前進，並抑制震盪 0.9
MOMENTUM = None

# (Float)優化器模糊因子比率 1e-7
EPSILON = None

# (String)優化器指定，None為客制，須另外撰寫
BASE_OPTIMIZERS = optimizers.Adam()

# (Float)標籤平滑比率，將one-hot的編碼方式變得更加soft 0.1
LABEL_SMOOTHING = 0

# (String)損失函數，None為客制，須另外撰寫
BASE_LOSSES = None

# (Float)focusing parameter for modulating factor (1-p)
LOSSES_GAMMA = 2.

# (Float)the same as weighing factor in balanced cross entropy
LOSSES_ALPHA = .25

# (String List)評價指標，None為客制，須另外撰寫
BASE_METRICS = ['accuracy']

# (String )評價指標的圖表顯示
PLOT_METRICS = 'accuracy'


''''訓練參數設定'''

# (Int List)每批訓練的尺寸
BATCH_SIZE = [16]*FOLD

# (Int)訓練做幾次時代
EPOCHS = [2]*FOLD

# (Int)從多少時代開始訓練
INITIAL_EPOCH = 0

# (Int)日誌顯示，0為靜音，1為進度條，2為顯示每個紀錄
VERBOSE = 1

# (Int)使用基於進程的線程時，要啟動的最大進程數。如果未指定，workers則默認為1。如果為0，將在主線程上執行生成器
WORKERS = 1

# (Boolean)如果為True，請使用基於多進程的線程。如果未指定，則默認為False
# 請注意，由於此實現依賴於多處理，因此不應將不可拾取的參數傳遞給生成器，因為它們無法輕易傳遞給子進程
USE_MULTIPROCESS = False

# (Boolean) 是否使用class_weight來平衡訓練標籤的分配不均，可能會降低準確率，不過會提昇真實標籤的正確性。
USE_ClASS_WEIGHT = True


''''圖表參數設定'''

# (Float)全部SNS圖表的字形縮放
ALL_SNS_FONT_SCALE = 1.0

# (Int)CSV缺失值圖表寬度
CSV_COUNTPLOT_FIGSIZE_W = 10

# (Int)CSV缺失值圖表高度
CSV_COUNTPLOT_FIGSIZE_H = 10

# (Int)CSV缺失值圖表標題字型大小
CSV_COUNTPLOT_TITLE_FONTSIZE = 20

# (Int)CSV缺失值圖表X軸標題字型大小
CSV_COUNTPLOT_XLABEL_FONTSIZE = 15

# (Int)CSV缺失值圖表Y軸標題字型大小
CSV_COUNTPLOT_YLABEL_FONTSIZE = 15

# (Int)訓練歷程圖表寬度
TRAINING_CURVES_FIGSIZE_W = 20

# (Int)訓練歷程圖表高度
TRAINING_CURVES_FIGSIZE_H = 10

# (Int)訓練歷程圖表SCATTER的標記點大小 
TRAINING_CURVES_SCATTER_SCALAR = 200

# (Float)訓練歷程圖表SCATTER的指標文字離標記點X距離係數
TRAINING_CURVES_SCATTER_METRICS_TEXT_XSCALAR = 0.03

# (Float)訓練歷程圖表SCATTER的指標文字標記點Y距離係數
TRAINING_CURVES_SCATTER_METRICS_TEXT_YSCALAR = 0.13

# (Float)訓練歷程圖表SCATTER的損失文字標記點X距離係數
TRAINING_CURVES_SCATTER_LOSS_TEXT_XSCALAR = 0.03

# (Float)訓練歷程圖表SCATTER的損失文字標記點Y距離係數
TRAINING_CURVES_SCATTER_LOSS_TEXT_YSCALAR = 0.05

# (Int)訓練歷程圖表SCATTER的文字大小
TRAINING_CURVES_SCATTER_TEXTSIZE = 15

# (Int)訓練歷程圖表X軸標題字型大小
TRAINING_CURVES_XLABEL_FONTSIZE = 15

# (Int)訓練歷程圖表Y軸標題字型大小
TRAINING_CURVES_YLABEL_FONTSIZE = 15

# (Int)訓練歷程圖表標題字型大小
TRAINING_CURVES_TITLE_FONTSIZE = 20

# (Float)訓練歷程圖表格線粗度
TRAINING_CURVES_GRID_ALPHA = 0

# (Int)混淆矩陣圖表寬度
CONFUSION_MATRIX_FIGSIZE_W = 10

# (Int)混淆矩陣圖表高度
CONFUSION_MATRIX_FIGSIZE_H = 10

# (Int)混淆矩陣圖表內容字型大小
CONFUSION_MATRIX_HEATMAP_FONTSIZE = 15

# (Int)混淆矩陣圖表標題字型大小
CONFUSION_MATRIX_TITLE_FONTSIZE = 20

# (Int)混淆矩陣圖表X軸標題字型大小
CONFUSION_MATRIX_XLABEL_FONTSIZE = 15

# (Int)混淆矩陣圖表Y軸標題字型大小
CONFUSION_MATRIX_YLABEL_FONTSIZE = 15

# (String)預設'binary'：僅在目標為二進制，兩分類時適用。
#'micro'：通過計算總的真陽性，假陰性和假陽性來全局計算指標，多分類時適用。
#'macro'：計算每個標籤的指標，並找到其未加權平均值。這沒有考慮標籤不平衡，多分類時適用。
AVERAGE = 'macro'

In [ ]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    
seed_everything(SEED)

In [ ]:
# 設置sns圖表縮放係數
sns.set(font_scale = ALL_SNS_FONT_SCALE)

# 4. 資料處理<a class="anchor" id="4"></a>
[Back to Table of Contents](#0)

## 4.1 載入CSV檔 <a class="anchor" id="4.1"></a>
[Back to Table of Contents](#0)

In [ ]:
if LOAD_CSV:
    print('Reading data...')

    # 讀取訓練資料集CSV檔
    train_csv = pd.read_csv(TRAIN_CSV_PATH,encoding="utf8")

    print('Reading data completed')

In [ ]:
if LOAD_CSV:
    # 顯示訓練資料集CSV檔
    print(train_csv.head())

In [ ]:
if LOAD_CSV:
    print("Shape of train_data :", train_csv.shape)

## 4.2 檢查CSV檔缺失值 <a class="anchor" id="4.2"></a>
[Back to Table of Contents](#0)

In [ ]:
if LOAD_CSV:
    total = train_csv.isnull().sum().sort_values(ascending = False)
    percent = (train_csv.isnull().sum()/train_csv.isnull().count()*100).sort_values(ascending = False)
    missing_train_csv  = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    print(missing_train_csv.head())

In [ ]:
if LOAD_CSV:
    print(train_csv[LABEL_NAME].value_counts())
    f,ax = plt.subplots(figsize=(CSV_COUNTPLOT_FIGSIZE_W, CSV_COUNTPLOT_FIGSIZE_H))
    sns.countplot(train_csv[LABEL_NAME], hue = train_csv[LABEL_NAME],ax = ax)
    plt.title("LABEL COUNT", fontsize=CSV_COUNTPLOT_TITLE_FONTSIZE)
    plt.xlabel(LABEL_NAME.upper(), fontsize=CSV_COUNTPLOT_XLABEL_FONTSIZE)
    plt.ylabel("COUNT", fontsize=CSV_COUNTPLOT_YLABEL_FONTSIZE)
    plt.legend()
    plt.show()

# 5. 定義模型方法<a class="anchor" id="5"></a>
[Back to Table of Contents](#0)

In [ ]:
def binary_focal_loss(gamma=2., alpha=.25):
    """
    Binary form of focal loss.
      FL(p_t) = -alpha * (1 - p_t)**gamma * log(p_t)
      where p = sigmoid(x), p_t = p or 1 - p depending on if the label is 1 or 0, respectively.
    References:
        https://arxiv.org/pdf/1708.02002.pdf
    Usage:
     model.compile(loss=[binary_focal_loss(alpha=.25, gamma=2)], metrics=["accuracy"], optimizer=adam)
    """
    def binary_focal_loss_fixed(y_true, y_pred):
        """
        :param y_true: A tensor of the same shape as `y_pred`
        :param y_pred:  A tensor resulting from a sigmoid
        :return: Output tensor.
        """
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))

        epsilon = K.epsilon()
        # clip to prevent NaN's and Inf's
        pt_1 = K.clip(pt_1, epsilon, 1. - epsilon)
        pt_0 = K.clip(pt_0, epsilon, 1. - epsilon)

        return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1)) \
               -K.sum((1 - alpha) * K.pow(pt_0, gamma) * K.log(1. - pt_0))

    return binary_focal_loss_fixed


def categorical_focal_loss(gamma=2., alpha=.25):
    """
    Softmax version of focal loss.
           m
      FL = ∑  -alpha * (1 - p_o,c)^gamma * y_o,c * log(p_o,c)
          c=1
      where m = number of classes, c = class and o = observation
    Parameters:
      alpha -- the same as weighing factor in balanced cross entropy
      gamma -- focusing parameter for modulating factor (1-p)
    Default value:
      gamma -- 2.0 as mentioned in the paper
      alpha -- 0.25 as mentioned in the paper
    References:
        Official paper: https://arxiv.org/pdf/1708.02002.pdf
        https://www.tensorflow.org/api_docs/python/tf/keras/backend/categorical_crossentropy
    Usage:
     model.compile(loss=[categorical_focal_loss(alpha=.25, gamma=2)], metrics=["accuracy"], optimizer=adam)
    """
    def categorical_focal_loss_fixed(y_true, y_pred):
        """
        :param y_true: A tensor of the same shape as `y_pred`
        :param y_pred: A tensor resulting from a softmax
        :return: Output tensor.
        """

        # Scale predictions so that the class probas of each sample sum to 1
        y_pred /= K.sum(y_pred, axis=-1, keepdims=True)

        # Clip the prediction value to prevent NaN's and Inf's
        epsilon = K.epsilon()
        y_pred = K.clip(y_pred, epsilon, 1. - epsilon)

        # Calculate Cross Entropy
        cross_entropy = -y_true * K.log(y_pred)

        # Calculate Focal Loss
        loss = alpha * K.pow(1 - y_pred, gamma) * cross_entropy

        # Sum the losses in mini_batch
        return K.sum(loss, axis=1)

    return categorical_focal_loss_fixed

In [ ]:
def build_optimizers():
    if BASE_OPTIMIZERS == None:
        print("Custiom OPTIMIZERS")
    else:
        RETURN_OPTIMIZERS = BASE_OPTIMIZERS
    return RETURN_OPTIMIZERS

optimizer = build_optimizers()

In [ ]:
def build_losses():
    if BASE_LOSSES == None:
        if CLASSES == 2:
            RETURN_LOSSES = binary_focal_loss(gamma = LOSSES_GAMMA, alpha = LOSSES_ALPHA)
        else:
            RETURN_LOSSES = categorical_focal_loss(gamma = LOSSES_GAMMA, alpha = LOSSES_ALPHA)
        print("Custiom LOSSES")
    else:
        RETURN_LOSSES = BASE_LOSSES
    return RETURN_LOSSES

loss = build_losses()

In [ ]:
def build_metrics():
    if BASE_METRICS == None:
        print("Custiom METRICS")
    else:
        RETURN_METRICS = BASE_METRICS
    return RETURN_METRICS

metrics = build_metrics()

In [ ]:
def build_model(dim = 256, fold = 0):
    if LOAD_MODEL:
        print("Reading the pre-trained model... ")
        model = keras.models.load_model(LOAD_MODEL_PATH)
        print("Reading done. ")
    else:
        if USE_BASE_MODEL:
            if USE_EFFICIENTNET_MODEL:
                base_model = BASE_MODEL[EFF_NET[fold]](input_shape=(dim,dim,3), include_top=INCLUDE_TOP, weights=WEIGHTS)
                for layer in base_model.layers:
                    layer.trainable = BASE_MODEL_TRAINABLE
                inputs = base_model.inputs[0]
                outputs = base_model.outputs[0]
                x = layers.AveragePooling2D(name="averagepooling2d_head")(outputs)
                x = layers.Flatten(name="flatten_head")(x)
                x = layers.Dense(64, activation="relu", name="dense_head")(x)
                x = layers.Dropout(DROPOUT, name="dropout_head")(x)
            else:
                base_model = BASE_MODEL(input_shape =(dim, dim, 3), include_top=INCLUDE_TOP, weights=WEIGHTS)
                for layer in base_model.layers:
                    layer.trainable = BASE_MODEL_TRAINABLE
                inputs = base_model.inputs[0]
                outputs = base_model.outputs[0]
                x = layers.GlobalAveragePooling2D(name="globalaveragepooling2d_head")(outputs)
                x = layers.Dropout(DROPOUT, name="dropout_head")(x)
        else:
            print("Custiom Model")
        outputs = layers.Dense(CLASSES, activation=ACTIVATION_FUNCTION, name="predictions_head")(x)                                    
        model = Model(inputs, outputs)
        model.compile(optimizer = optimizer, loss = loss, metrics = metrics)

        if MODEL_PRINT:
            model.summary()
        if SAVE_MODEL_DIAGRAM:
            plot_model(model, show_shapes= SAVE_MODEL_SHAPE, to_file=SAVE_MODEL_FILENAME)

        if LOAD_WEIGHTS :
            print("Reading the pre-trained weights... ")
            model.load_weights(LOAD_WEIGHTS_PATH)
            print("Reading done. ")
    return model

# 6. 定義回調函數方法<a class="anchor" id="6"></a>
[Back to Table of Contents](#0)

In [ ]:
def get_callbacks(checkpointpath):
    check_pointer = callbacks.ModelCheckpoint(filepath = checkpointpath, monitor = MONITOR, verbose = VERBOSE, 
                                              save_best_only = SAVE_BEST_ONLY)

    # Interrupt the training when the validation loss is not decreasing
    early_stopping = callbacks.EarlyStopping(monitor = MONITOR, verbose = VERBOSE, patience = PATIENCE_ELS)

    # Stream each epoch results into a .csv file
    csv_logger = callbacks.CSVLogger(CSV_SAVE_PATH, separator = ',', append = APPEND)

    # LEARNING RATE SCHEDULER
    def get_lr_callback():
        def lrfn(epoch):
            if epoch < LEARNING_RATE_RAMPUP_EPOCHS:
                lr = (LEARNING_RATE_MAX - LEARNING_RATE_START) / LEARNING_RATE_RAMPUP_EPOCHS * epoch + LEARNING_RATE_START

            elif epoch < LEARNING_RATE_RAMPUP_EPOCHS + LEARNING_RATE_SUSTAIN_EPOCHS:
                lr = LEARNING_RATE_MAX

            else:
                lr = (LEARNING_RATE_MAX - LEARNING_RATE_MIN) * LEARNING_RATE_EXP_DECAY**(epoch - LEARNING_RATE_RAMPUP_EPOCHS - LEARNING_RATE_SUSTAIN_EPOCHS) + LEARNING_RATE_MIN

            return lr

        lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose=VERBOSE)
        return lr_callback

    lr_scheduler = get_lr_callback()

    # Reduce learning rate when a metric has stopped improving
    reduce_lr = callbacks.ReduceLROnPlateau(monitor = MONITOR, verbose = VERBOSE, factor = FACTOR, patience = PATIENCE_RLR, 
                                  min_lr = LEARNING_RATE_MIN_RLR)

    tensor_board = callbacks.TensorBoard(log_dir = TENSORBOARD_LOGS_PATH, update_freq = UPDATE_FREQ, write_graph = WRITE_GRAPH, 
                               profile_batch = PROFILE_BATCH)

    callbacks_list = []
    if CALLBACKS_CHECK_POINTER:
        callbacks_list.append(check_pointer)
    if CALLBACKS_EARLY_STOPPING:
        callbacks_list.append(early_stopping)
    if CALLBACKS_CSV_LOGGER:
        callbacks_list.append(csv_logger)
    if CALLBACKS_LR_SCHEDULER:
        callbacks_list.append(lr_scheduler)
    if CALLBACKS_REDUCE_LR:
        callbacks_list.append(reduce_lr)
    if CALLBACKS_TENSOR_BOARD:
        callbacks_list.append(tensor_board)
    
    return callbacks_list

# 7. 製作資料集＆資料擴增&訓練模型 <a class="anchor" id="7"></a>
[Back to Table of Contents](#0)

In [ ]:
# 宣告為訓練後預測用
all_labels = []; all_pred = []

In [ ]:
def display_training_curves(history, fold, skf):
    plt.figure(figsize=(TRAINING_CURVES_FIGSIZE_W,TRAINING_CURVES_FIGSIZE_H))
    plt.plot(np.arange(EPOCHS[fold]),history.history[PLOT_METRICS],'-o',label='TRAIN '+PLOT_METRICS.upper(),color='#ff7f0e')
    plt.plot(np.arange(EPOCHS[fold]),history.history['val_'+PLOT_METRICS],'-o',label='VALIDATION '+PLOT_METRICS.upper(),color='#1f77b4')
    x = np.argmax( history.history['val_'+PLOT_METRICS] ); y = np.max( history.history['val_'+PLOT_METRICS] )
    xdist = plt.xlim()[1] - plt.xlim()[0]; ydist = plt.ylim()[1] - plt.ylim()[0]
    plt.scatter(x,y,s=TRAINING_CURVES_SCATTER_SCALAR,color='#1f77b4')
    plt.text(x-TRAINING_CURVES_SCATTER_METRICS_TEXT_XSCALAR*xdist,y-TRAINING_CURVES_SCATTER_METRICS_TEXT_YSCALAR*ydist,'max '+PLOT_METRICS+'\n%.4f'%y,size=TRAINING_CURVES_SCATTER_TEXTSIZE)
    plt.ylabel(PLOT_METRICS.upper(),size=TRAINING_CURVES_YLABEL_FONTSIZE); plt.xlabel('EPOCH',size=TRAINING_CURVES_XLABEL_FONTSIZE)
    plt.grid(alpha=TRAINING_CURVES_GRID_ALPHA)
    plt.legend(loc=2)
    plt2 = plt.gca().twinx()
    plt2.plot(np.arange(EPOCHS[fold]),history.history['loss'],'-o',label='TRAIN LOSS',color='#2ca02c')
    plt2.plot(np.arange(EPOCHS[fold]),history.history['val_loss'],'-o',label='VALIDATION LOSS',color='#d62728')
    x = np.argmin( history.history['val_loss'] ); y = np.min( history.history['val_loss'] )
    ydist = plt.ylim()[1] - plt.ylim()[0]
    plt.scatter(x,y,s=TRAINING_CURVES_SCATTER_SCALAR,color='#d62728')
    plt.text(x-TRAINING_CURVES_SCATTER_LOSS_TEXT_XSCALAR*xdist,y+TRAINING_CURVES_SCATTER_LOSS_TEXT_YSCALAR*ydist,'min loss\n%.4f'%y,size=TRAINING_CURVES_SCATTER_TEXTSIZE)
    plt.ylabel('LOSS',size=TRAINING_CURVES_YLABEL_FONTSIZE)
    if skf:
        plt.title('FOLD %i - IMAGE SIZE %i, %s'%
                  (fold+1, IMAGE_SIZE[fold], MODEL_NAME.upper()), size=TRAINING_CURVES_TITLE_FONTSIZE)
    else:
        plt.title(' IMAGE SIZE %i, %s'%
                  (IMAGE_SIZE[fold], MODEL_NAME.upper()), size=TRAINING_CURVES_TITLE_FONTSIZE)
    plt.grid(alpha=TRAINING_CURVES_GRID_ALPHA)
    plt.legend(loc=3)
    plt.show()

In [ ]:
def train_process(fold, skf, x_train, x_val, y_train, y_val):
    if skf:
        print('FOLD %i - IMAGE SIZE %i WITH %s AND BATCH_SIZE %i'%(fold+1,IMAGE_SIZE[fold],MODEL_NAME.upper(),BATCH_SIZE[fold]*REPLICAS))
    else:
        print('IMAGE SIZE %i WITH %s AND BATCH_SIZE %i'%(IMAGE_SIZE[fold],MODEL_NAME.upper(),BATCH_SIZE[fold]*REPLICAS))
        
    if LOAD_CSV:
        train_data = pd.DataFrame(x_train)
        train_data.columns = [IMAGE_NAME_ROOT]
        train_data[LABEL_NAME] = y_train

        validation_data = pd.DataFrame(x_val)
        validation_data.columns = [IMAGE_NAME_ROOT]
        validation_data[LABEL_NAME] = y_val

        train_data[LABEL_NAME] = train_data[LABEL_NAME].astype(LABEL_NAME_TYPE)
        validation_data[LABEL_NAME] = validation_data[LABEL_NAME].astype(LABEL_NAME_TYPE)

        train_datagen = ImageDataGenerator(rescale = 1. / 255,
                                           zca_whitening = ZCA_WHITENING,
                                           rotation_range = ROTATION_RANGE,
                                           width_shift_range = WIDTH_SHIFT_RANGE,
                                           height_shift_range = HEIGHT_SHIFT_RANGE,
                                           shear_range = SHEAR_RANGE,
                                           zoom_range = ZOOM_RANGE,
                                           horizontal_flip = HORIZONTAL_FILP,
                                           vertical_flip = VERTICAL_FILP,
                                           fill_mode = FILL_MODE)

        val_datagen = ImageDataGenerator(rescale = 1./255)

        train_generator = train_datagen.flow_from_dataframe(
            train_data,
            x_col = IMAGE_NAME_ROOT,
            y_col = LABEL_NAME,
            target_size = (IMAGE_SIZE[fold],IMAGE_SIZE[fold]),
            batch_size = BATCH_SIZE[fold],
            shuffle = True,
            seed = SEED,
            color_mode = COLOR_MODE,
            class_mode = CLASS_MODE)

        validation_generator = val_datagen.flow_from_dataframe(
            validation_data,
            x_col = IMAGE_NAME_ROOT,
            y_col = LABEL_NAME,
            target_size = (IMAGE_SIZE[fold], IMAGE_SIZE[fold]),
            batch_size = BATCH_SIZE[fold],
            shuffle = False,
            seed = SEED,
            color_mode = COLOR_MODE,
            class_mode = CLASS_MODE)
    else:
        train_datagen = ImageDataGenerator(rescale = 1. / 255,
                                           zca_whitening = ZCA_WHITENING,
                                           rotation_range = ROTATION_RANGE,
                                           width_shift_range = WIDTH_SHIFT_RANGE,
                                           height_shift_range = HEIGHT_SHIFT_RANGE,
                                           shear_range = SHEAR_RANGE,
                                           zoom_range = ZOOM_RANGE,
                                           horizontal_flip = HORIZONTAL_FILP,
                                           vertical_flip = VERTICAL_FILP,
                                           fill_mode = FILL_MODE,
                                           validation_split = DATA_SPLIT)
        
        validation_datagen = ImageDataGenerator(rescale = 1. / 255,
                                           validation_split = DATA_SPLIT)

        train_generator = train_datagen.flow_from_directory(
                    TRAIN_DATA_PATH, 
                    target_size = (IMAGE_SIZE[fold], IMAGE_SIZE[fold]), 
                    batch_size = BATCH_SIZE[fold], 
                    shuffle = True,
                    seed = SEED,
                    color_mode = COLOR_MODE, 
                    class_mode = CLASS_MODE, 
                    subset = 'training')

        validation_generator = validation_datagen.flow_from_directory(
                    TRAIN_DATA_PATH, 
                    target_size = (IMAGE_SIZE[fold], IMAGE_SIZE[fold]), 
                    batch_size = BATCH_SIZE[fold], 
                    shuffle = False,
                    seed = SEED,
                    color_mode = COLOR_MODE, 
                    class_mode = CLASS_MODE, 
                    subset = 'validation')
        
    # (Int)聲明一個紀元完成並開始下一個紀元之前的總步數(一批樣品)
    STEPS_PER_EPOCH = tf.math.ceil(train_generator.n/train_generator.batch_size/REPLICAS)
    # (Int)在每個時期結束時執行驗證時，在停止之前要繪製的步驟總數(樣本批次)
    VALIDATION_STEPS = tf.math.ceil(validation_generator.n/validation_generator.batch_size/REPLICAS)
    print("Number of training and validation steps: {} and {}".format(STEPS_PER_EPOCH,VALIDATION_STEPS))
     
    # BUILD MODEL
    K.clear_session()
    with strategy.scope():
        model = build_model(dim = IMAGE_SIZE[fold], fold = fold)
     
    if LOAD_CSV and skf:
        # (String)訓練模型FOLD>1的儲存路徑
        SAVE_MODEL_PATH = PROJECT_PATH+r'/models/'+MODEL_NAME+'_fold_%i.h5'%(fold+1)
    else:
        SAVE_MODEL_PATH = TRAIN_MODEL_PATH
        
    print("train batch ", train_generator.__getitem__(0)[0].shape)
    print("validation batch ", validation_generator.__getitem__(0)[0].shape)
    print("sample train label \n", train_generator.__getitem__(0)[1][:5])
    print("sample validation label \n", validation_generator.__getitem__(0)[1][:5])
    print("train class indices ", train_generator.class_indices)
    print("validation class indices ", validation_generator.class_indices)
        
    counter = Counter(train_generator.classes)
    max_val = float(max(counter.values()))
    class_weight = {class_id : max_val/num_images for class_id, num_images in counter.items()}
    print('calss weight ', class_weight)
    if not USE_ClASS_WEIGHT:
        class_weight = None
        
    print('Training...')
    history = model.fit(
        train_generator, 
        steps_per_epoch = STEPS_PER_EPOCH, 
        epochs = EPOCHS[fold], 
        callbacks = get_callbacks(checkpointpath = SAVE_MODEL_PATH), 
        validation_data = validation_generator, 
        validation_steps = VALIDATION_STEPS, 
        verbose = VERBOSE, 
        workers = WORKERS, 
        use_multiprocessing = USE_MULTIPROCESS, 
        initial_epoch = INITIAL_EPOCH, 
        class_weight = class_weight)
    print('Training done!')
    
    if CALLBACKS_CHECK_POINTER:
        model.load_weights(TRAIN_MODEL_PATH)
    else:
        model.save(TRAIN_MODEL_PATH)
        
    display_training_curves(history, fold, skf)
        
    all_labels.append(validation_generator.classes)
    all_pred.append(np.argmax(model.predict(validation_generator), axis = -1))
    
    if LOAD_CSV:
        del x_train, x_val, y_train, y_val
    del train_data, validation_data
    del train_generator, validation_generator, history
    if LOAD_CSV and skf:
        del model
    K.clear_session()
    gc.collect()

In [ ]:
if LOAD_CSV:
    label_list = []
    train_list = []
    for i in range(train_csv.shape[0]):
        if IMAGE_NAME_HAVE_EXTENSION:
            train_list.append(TRAIN_DATA_PATH + '/' + train_csv[IMAGE_NAME].iloc[i])
        else:
            train_list.append(TRAIN_DATA_PATH + '/' + train_csv[IMAGE_NAME].iloc[i] + IMAGE_NAME_EXTENSION)
        label_list.append(train_csv[LABEL_NAME].iloc[i])
    df_train = pd.DataFrame(train_list)
    df_train.columns = [IMAGE_NAME_ROOT]
    df_train[LABEL_NAME] = label_list

In [ ]:
%%time
if LOAD_CSV and FOLD == 1:
    X_train, X_val, Y_train, Y_val = train_test_split(df_train[IMAGE_NAME_ROOT],df_train[LABEL_NAME], test_size = DATA_SPLIT, random_state = SEED)
    train_process(fold = 0, skf = False, x_train = X_train, x_val = X_val, y_train = Y_train, y_val = Y_val)
    del X_train, X_val, Y_train, Y_val
    K.clear_session()
    gc.collect()
elif LOAD_CSV and FOLD > 1:
    for fold,(train_index, val_index) in enumerate(SKF.split(df_train[IMAGE_NAME_ROOT], df_train[LABEL_NAME])):
        X_train, X_val = df_train[IMAGE_NAME_ROOT].iloc[train_index], df_train[IMAGE_NAME_ROOT].iloc[val_index]
        Y_train, Y_val = df_train[LABEL_NAME].iloc[train_index], df_train[LABEL_NAME].iloc[val_index]
        train_process(fold = fold, skf = True, x_train = X_train, x_val = X_val, y_train = Y_train, y_val = Y_val)
        del X_train, X_val, Y_train, Y_val
        K.clear_session()
        gc.collect()
else:
    train_process(fold = 0, skf = False)

# 8. 混淆矩陣<a class="anchor" id="8"></a>
[Back to Table of Contents](#0)

In [ ]:
cm_correct_labels = np.concatenate(all_labels)
cm_predictions = np.concatenate(all_pred)
print("Correct   labels: ", cm_correct_labels.shape, cm_correct_labels)
print("Predicted labels: ", cm_predictions.shape, cm_predictions)

In [ ]:
'''混淆矩陣包含四個要素:TP(True Positive)正確預測成功的正樣本, TN(True Negative)正確預測成功的負樣本, 
FP(False Positive)錯誤預測成正樣本，實際上為負樣本, FN(False Negative)錯誤預測成負樣本(或者說沒能預測出來的正樣本)'''
cm = tf.math.confusion_matrix(cm_correct_labels, cm_predictions)
cm = cm/cm.numpy().sum(axis=1)[:, tf.newaxis]

f,ax = plt.subplots(figsize = (CONFUSION_MATRIX_FIGSIZE_W, CONFUSION_MATRIX_FIGSIZE_H))
sns.heatmap(cm, annot = True, ax = ax, annot_kws={"size": CONFUSION_MATRIX_HEATMAP_FONTSIZE})
plt.title("CONFUSION MATRIX", fontsize=CONFUSION_MATRIX_TITLE_FONTSIZE)
plt.xlabel("PREDICTED", fontsize=CONFUSION_MATRIX_XLABEL_FONTSIZE)
plt.ylabel("TRUE", fontsize=CONFUSION_MATRIX_YLABEL_FONTSIZE)
plt.show()

In [ ]:
'''
Accuracy = (TP+TN)/(TP+FP+TN+FN)
Precision(準確率) = TP/(TP+FP)
Recall(召回率) = TP/(TP+FN)
F1-score(Recall與Precision的調和平均數) = 2 * Precision * Recall / (Precision + Recall)
'''
accuracy = accuracy_score(cm_correct_labels, cm_predictions)
precision = precision_score(cm_correct_labels, cm_predictions, labels = range(CLASSES), average = AVERAGE)
recall = recall_score(cm_correct_labels, cm_predictions, labels = range(CLASSES), average = AVERAGE)
score = f1_score( cm_correct_labels, cm_predictions, labels = range(CLASSES), average = AVERAGE)
print('Accuracy: {:.3f}, Precision: {:.3f}, Recall: {:.3f}, F1 score: {:.3f}'.format(accuracy, precision, recall, score))

In [ ]:
# Classification report on training Data
print(classification_report(cm_correct_labels, cm_predictions))

# 9. 待辦事項<a class="anchor" id="9"></a>

[Back to Table of Contents](#0)

[Go to Top](#0)